# TRANSLATION WITH A SEQUENCE TO SEQUENCE NETWORK AND ATTENTION

In [1]:
%matplotlib inline
import warnings
warnings.simplefilter('ignore')

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## The Lang Class

In [3]:
SOS_token = 0
EOS_token = 1

class Lang:
    
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {
            0:"SOS",
            1:"EOS"
        }
        self.n_words = 2 # Just SOS and EOS
        
    def addSentence(self, sentence):
        """
        adds each word of the sentence in vocab
        
        :param sentence: a sentence of words
        """
        
        for word in sentence.split(' '):
            self.addWord(word)
            
    def addWord(self, word):
        """
        adds the word in vocab
        
        :param word: the word to be added
        """
        
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
            
        else:
            self.word2count[word] += 1

## Some preprocessing Utils

In [4]:
# Turn a unicode string to plain ASCII

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category != 'Mn'
    )


# Lowercase, trim, and remove non-letter characters

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

**To read the data file we will split the file into lines, and then split lines into pairs. The files are all English → Other Language, so if we want to translate from Other Language → English. The reverse flag is used to reverse the pairs.**

In [5]:
def readLangs(lang1="eng", lang2="fra", reverse=False):
    """
    
    :param lang1: the language to be translated.
    :param lang2: the language to be translated to.
    :param reverse: reverse the scenario
    
    What this function does is, it will return the instances
    of `Lang` class for each language and the corresponding
    lang1-lang2 pairs.
    
    like :
    Eng - French pairs
    "I am cold.   J'ai froid." => ["I am cold.", "J'ai froid."]
    
    """
    
    print("Reading Lines...")
    
    # Read the file and split into lines
    lines = open(f"../input/english-french-dataset/data/{lang1}-{lang2}.txt", 
                encoding='utf-8').read().strip().split('\n')
    
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
        
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
        
    return input_lang, output_lang, pairs

In [6]:
MAX_LENGTH = 10

# We are sampling some sentences so that, we can test the model fast
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    """
    a function to tell whether to,
    filter the given pair p.
    """
    
    return len(p[0].split(' ')) < MAX_LENGTH and \
            len(p[1].split(' ')) < MAX_LENGTH and \
            p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    """
    filter the given pairs and include if meets the criteria.
    """
    
    return [pair for pair in pairs if filterPair(pair)]

### The full process for preparing the data is:

* Read text file and split into lines, split lines into pairs
* Normalize text, filter by length and content
* Make word lists from sentences in pairs

In [7]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    
    print(f"Read {len(pairs)} sentenc pairs")
    
    pairs = filterPairs(pairs)
    print(f"Trimmed to {len(pairs)} sentence pairs")
    
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
        
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))
    

Reading Lines...
Read 135842 sentenc pairs
Trimmed to 9874 sentence pairs
Counting words...
Counted words:
fra 3802
eng 2689
['je t aide quand tu veux .', 'i am always ready to help you .']


## The Seq2Seq Model

### ENCODER CLASS

![](https://pytorch.org/tutorials/_images/encoder-network.png)

In [8]:
class EncoderRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        """
        :param input_size: the size of the vocab of 
            lang to be translated.
        :param hidden_size: the output dim of the emb vector
            also, the size of the hidden vector in RNN
        """
        
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        
    def forward(self, x, hidden):
        """
        :param x: the input of shape [batch_size, seq_len]
        :param hidden: the hidden matrix for RNN
        """
        
        embedded = self.embedding(x).view(1, 1, -1)
        # shape (1, 1, batch_size * seq_len * embedding_size)
        
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## DECODER CLASS
### (*simple decoder*)

![](https://pytorch.org/tutorials/_images/decoder-network.png)

In [9]:
class DecoderRNN(nn.Module):
    
    def __init__(self, hidden_size, output_size):
        """
        :param hidden_size: the output dim of the emb vector
            also, the size of the hidden vector in RNN
        :param output_size: the size of the vocab of 
            lang to be translated to.
        """
        
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hiddem_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, hidden):
        """
        :param x: the input of shape [batch_size, seq_len]
        :param hidden: the hidden matrix for RNN
        """
        
        output = self.embedding(x).view(1, 1, -1)
        # shape (1, 1, batch_size * seq_len * embedding_size)
        
        output = F.relu(output)
        
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
        

## *ATTENTION* DECODER CLASS

![](https://pytorch.org/tutorials/_images/attention-decoder-network.png)

In [10]:
class AttnDecoderRNN(nn.Module):
    
    def __init__(self, hidden_size, output_size, 
                 dropout_p=0.1, max_length=MAX_LENGTH):
        """
        :param hidden_size: the hidden matrix for RNN
        :param output_size: the size of the vocab of 
            lang to be translated to.
        :param dropout_p: dropout probability
        :param max_length: the maximum length of the sequence
        """
        
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        
        
        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, x, hidden, encoder_outputs):
        """
        :param x: the input tensor for the decoder (used only for training)
        :param hidden: the hidden context vectors
        :param encoder_outputs: the encoder outputs for the sentence
        """
        
        embedded = self.embedding(x).view(1, 1, -1)
        # shape (1, 1, batch_size * seq_len * embedding_size)
        
        embedded = self.dropout(embedded)
        
        # Concatenating the previous hidden and the encoder outputs
        # Then calculating the `attention weights`
        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)),
            dim=1
        )
        
        # Applying attention: multiplying the corresponding attention
        # with the encoder output
        attn_applied = torch.bmm(
            attn_weights.unsqueeze(0),
            encoder_outputs.unsqueeze(0)
        )
        
        
        # Concatenating the ``Word to be predicted`` embedding
        # with the ``attention applied`` encoder vectors
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)
        
        output = F.relu(output)
        
        # at first, this hidden is the context vector from the encoder
        # after that, we will pass this hidden, as the ``decoder hidden`` context vector
        output, hidden = self.gru(output, hidden)
        
        # This output goes towards predicting the required translated word
        output = F.log_softmax(self.out(output[0]), dim=1)
        
        return output, hidden, attn_weights
        
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# Training

## Preparing Training Data

In [11]:
def indexesFromSentence(lang, sentence):
    """
    converts the words of a sentence to a list of indexes
    """
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    """
    converts the sentence to it's corresponding Tensor
    """
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    """
    transform the sentences in the pair
    
    e.g:
        ["I am cold.", "J'ai froid."] --> ([34, 88, 91], [22, 45, 97])
    """
    
    input_tensor = tensorFromSentence(input_lang, pair[0])
    output_tensor = tensorFromSentence(output_lang, pair[1])
    
    return (input_tensor, output_tensor)

## Training the Model

In [12]:
teacher_force_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, 
          encoder_optimizer, decoder_optimizer, 
          criterion, max_length=MAX_LENGTH):
    
    """
    :param input_tensor: the input sequence
    :param target_tensor: the target sequence
    :param encoder: the Encoder
    :param decoder: the Decoder
    :param encoder_optimizer: the optimizer used for Encoder
    :param decoder_optimizer: the optimizer used for Decoder
    :param criterion: the loss function used
    :param max_length: the max length of the sequence
    """
    
    encoder_hidden = encoder.initHidden()
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    
    loss = 0
    
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden
        )
        
        encoder_outputs[ei] = encoder_output[0, 0]
        
    decoder_input = torch.tensor([[SOS_token]], device=device)
    
    decoder_hidden = encoder_hidden
    
    use_teacher_forcing = True if random.random() < teacher_force_ratio else False
        
    if use_teacher_forcing:
        
        # Feed the target as the next input
        
        for di in range(target_length):
            
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]
            
    else:
        
        # Use own prediction as the next input
        
        for di in range(target_length):
            
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            
            # topk : k largest
            topv, topi = decoder_output.topk(1)
            
            decoder_input = topi.squeeze().detach() # detach from history as input
            
            loss += criterion(decoder_output, target_tensor[di])
            
            if decoder_input.item() == EOS_token:
                break
                
    
    loss.backward()
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.item() / target_length

In [13]:
#  a helper function to print time elapsed and estimated time 
# remaining given the current time and progress %.

import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [14]:
def trainIters(encoder, decoder, n_iters, print_every=100, plot_every=100, 
               learning_rate=0.01):
    """
    :param encoder: the Encoder
    :param decoder: the Decoder
    :param n_iters: the number of epochs
    :param print_every: print every 100th result
    :param plot_every: plot every 100th result
    :param learning_rate: the learning rate to be applied
    """
    
    start = time.time()
    plot_losses = []
    
    print_loss_total = 0 # Reset at every print_every
    plot_loss_total = 0 # Reset at every plot_every
    
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                     for i in range(n_iters)]
    
    criterion = nn.NLLLoss()
    
    for iter in range(1, n_iters + 1):
        
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        
        loss = train(input_tensor, target_tensor, encoder, decoder, 
                     encoder_optimizer, decoder_optimizer, criterion)
        
        print_loss_total += loss
        plot_loss_total += loss
        
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
            
        
        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
            
    showPlot(plot_losses)

## Plotting Results

In [15]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    
    plt.figure()
    fig, ax = plt.subplots()
    
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

## Evaluation

In [16]:
def evaluate(encoder, decoder, sentence, 
             max_length=MAX_LENGTH):
    
    """
    :param encoder: the Encoder
    :param decoder: the Decoder
    :param sentence: the input sentence
    :param max_length: the max length of the sequence
    """
    
    with torch.no_grad():
        
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, 
                                      device=device)
        
        for ei in range(input_length):
            
            encoder_output, encoder_hidden = encoder(input_tensor[ei], 
                                                     encoder_hidden)
            
            encoder_outputs[ei] += encoder_output[0, 0]
            
        decoder_input = torch.tensor([[SOS_token]], device=device)
        
        decoder_hidden = encoder_hidden
        
        decoded_words = []
        
        decoder_attentions = torch.zeros(max_length, max_length)
        
        for di in range(max_length):
            
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            
            decoder_attentions[di] = decoder_attention.data
            
            topv, topi = decoder_output.data.topk(1)
            
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
                
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
                
            decoder_input = topi.squeeze().detach()
            
        return decoded_words, decoder_attentions[:di + 1]

In [17]:
# Evaluate Randomly
def evaluateRandomly(encoder, decoder, n=10):
    
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')
        

In [18]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 7500, print_every=5000)

2m 21s (- 1m 10s) (5000 66%) 2.8458


In [19]:
evaluateRandomly(encoder1, attn_decoder1)

> c est un jeune adolescent impressionnable .
= he s a young impressionable teenager .
< he s a bit good . . <EOS>

> elle est inquie te pour votre se curite .
= she s worried about your safety .
< she s very good for my . . <EOS>

> je vais vraiment manquer a tom .
= i m really going to miss tom .
< i m going to to the . . <EOS>

> elles sont dans le laboratoire de sciences .
= they re in the science lab .
< they are in the . <EOS>

> je suis au lit .
= i m in bed .
< i m a . . . <EOS>

> il est impitoyable .
= he s ruthless .
< he s in . <EOS>

> elle vous attend chez nous .
= she s waiting for you at home .
< she is going to . . . <EOS>

> vous e tes trop maigrichon .
= you re too skinny .
< you re very talented . <EOS>

> vous n e tes pas ainsi d ordinaire .
= you re not usually like this .
< you re not a . . . <EOS>

> je vais faire un feu .
= i m going to build a fire .
< i m going to . . . . <EOS>



## Visualising Attention

In [20]:
output_words, attentions = evaluate(
    encoder1, attn_decoder1, "je suis trop froid .")

plt.matshow(attentions.numpy())

In [21]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()
    
    
def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)


evaluateAndShowAttention("elle a cinq ans de moins que moi .")

evaluateAndShowAttention("elle est trop petit .")

evaluateAndShowAttention("je ne crains pas de mourir .")

evaluateAndShowAttention("c est un jeune directeur plein de talent .")

input = elle a cinq ans de moins que moi .
output = she s going to as . . <EOS>
input = elle est trop petit .
output = she is very for . . <EOS>
input = je ne crains pas de mourir .
output = i m not going . . . <EOS>
input = c est un jeune directeur plein de talent .
output = he s a bit and a . . <EOS>
